# Training Notebook

In this notebook, we will run models, also this notebook can be a template to run other models with different hyperparameters.

## Import libraries

In [4]:
from get_loader import get_loader
from models import Encoder, Decoder
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from data_prep_utils import *
from pathlib import Path
import json

## Load train and validation loaders

In [5]:
IMAGE_PATH = '../Datasets/coco/images/train2017'
CAPTIONS_PATH = '../Datasets/coco/annotations/' #captions_train2017.json'
#IMAGE_PATH = '../Datasets/coco/images/train2017'
#CAPTIONS_PATH = '../Datasets/coco/annotations/' #captions_train2017.json'
FREQ_THRESHOLD = 5
CAPS_PER_IMAGE = 5
SHUFFLE = True

# root of the name to save or load captions files
CAPTIONS_NAME = 'random_v2'

# for data loader
BATCH_SIZE = 128

In [6]:
# create custom data set if we need it. We can choose to work with certain types
# of images or reduce the size of the data
# this will write files to 'Datasets/coco/annotations' as 
#     [save_name]_captions_train.json
#     [save_name]_captions_val.json
#     [save_name]_captions_test.json

# prepare_datasets(train_percent = 0.87, super_categories=None,
#                  max_train=10000, max_val=2000, max_test=2000,
#                  save_name=CAPTIONS_NAME, random_seed=42)

# we explicitly build the vocab here. We use frequency threshold, and we build
# vocab from the specified captions file: we're using the training data
# we save the vocab to a name consistent with our training captions data so that 
# we can load a vocab consistent with the specific training run we've used.
build_vocab(freq_threshold = FREQ_THRESHOLD, 
            captions_file=f'{CAPTIONS_NAME}_captions_train.json',
            vocab_save_name=CAPTIONS_NAME)

There are 50026 captions in the data set
With FREQ_THRESHOLD = 5, vocab size is 3389


In [4]:
with open(f'../vocabulary/{CAPTIONS_NAME}word2idx.json', 'r') as f:
    word2idx = json.load(f)
vocab_size = len(word2idx)

In [5]:
train_loader_params = {
    'images_path': IMAGE_PATH,
    'captions_path': CAPTIONS_PATH + f'{CAPTIONS_NAME}_captions_train.json',
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': 5,
    'batch_size': BATCH_SIZE,
    'shuffle': SHUFFLE,
    'mode': 'train',
    # 'idx2word': None,
    'word2idx': word2idx
}

train_loader, train_dataset = get_loader(**train_loader_params)

val_loader_params = {
    'images_path': IMAGE_PATH,
    'captions_path': CAPTIONS_PATH + f'{CAPTIONS_NAME}_captions_val.json',
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': 3,
    'batch_size': BATCH_SIZE,
    'shuffle': SHUFFLE,
    'mode': 'validation',
    # 'idx2word': train_dataset.vocab.idx2word,
    'word2idx': word2idx
}

val_loader, val_dataset = get_loader(**val_loader_params)

print(f"Length of training dataloader: {len(train_loader)}, Length of testing dataloader: {len(val_loader)}")
print(f"Length of vocabulary: {len(train_dataset.vocab.idx2word)}")

Length of training dataloader: 391, Length of testing dataloader: 47
Length of vocabulary: 3389


## Load the model

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"We are using {device}.")

We are using cuda.


In [7]:
# for encoder and decoder
EMBED_SIZE = 1024 # dimension of vocab embedding vector
HIDDEN_SIZE = 512
NUM_LAYERS = 3 #hidden layers in LTSM
vocab_size = len(train_dataset.vocab.idx2word)

# training parameters
TOTAL_EPOCH = 50
CHECKPOINT = '../model/model_v2'
PRINT_EVERY = 50 # run print_every batches and then

In [8]:
model_params = {
    'save_path': CHECKPOINT,
    'batch_size': BATCH_SIZE,
    'embed_size': EMBED_SIZE,
    'hidden_size': HIDDEN_SIZE,
    'num_layers': NUM_LAYERS,
    'vocab_size': len(train_dataset.vocab.idx2word)
}

save_params(**model_params)

In [9]:
with open(os.path.join(CHECKPOINT, 'word2idx.json'), "w") as outfile:
    json.dump(word2idx, outfile)

## Training

In [10]:
encoder_ = Encoder(embed_size=EMBED_SIZE, pretrained=True)
decoder_ = Decoder(embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, vocab_size=vocab_size, num_layers=NUM_LAYERS)

In [11]:
# the loss is a cross entropy loss and ignore the index of <PAD> since it doesn't make any difference
criterion = nn.CrossEntropyLoss(ignore_index=train_dataset.vocab.word2idx["<PAD>"]).cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss(ignore_index=train_dataset.vocab.word2idx["<PAD>"])

# combine the parameters of decoder and encoder
params = list(decoder_.parameters()) + list(encoder_.embed.parameters())

# Adam optimizer
opt_pars = {'lr':3e-4, 'weight_decay':1e-3, 'betas':(0.9, 0.999), 'eps':1e-08}
optimizer = optim.Adam(params, **opt_pars)

In [12]:
train_params = {
    'encoder': encoder_,
    'decoder': decoder_,
    'criterion': criterion,
    'optimizer': optimizer,
    'train_loader': train_loader,
    'val_loader': val_loader,
    'total_epoch': TOTAL_EPOCH,
    'device': device,
    'checkpoint_path': CHECKPOINT,
    'print_every': PRINT_EVERY,
    'load_checkpoint': False
}

training_loss, validation_loss = train(**train_params) 

Epoch: [0/50]          || Step: [0/391]         || Average Training Loss: 8.1361
Epoch: [0/50]          || Step: [50/391]        || Average Training Loss: 6.1486
Epoch: [0/50]          || Step: [100/391]       || Average Training Loss: 5.4955
Epoch: [0/50]          || Step: [150/391]       || Average Training Loss: 5.2212
Epoch: [0/50]          || Step: [200/391]       || Average Training Loss: 5.0248
Epoch: [0/50]          || Step: [250/391]       || Average Training Loss: 4.8700
Epoch: [0/50]          || Step: [300/391]       || Average Training Loss: 4.7425
Epoch: [0/50]          || Step: [350/391]       || Average Training Loss: 4.6387
Epoch: [0/50]          || Step: [0/47]          || Average Validation Loss: 3.9465
****************************************************************************************************
Epoch: [0/50] || Training Loss = 4.57 || Validation Loss: 3.86 || Time: 19.594954
**************************************************************************************

KeyboardInterrupt: 